# Advanced Quantum Methods: Error Mitigation and Noise Handling

## Learning Objectives
- Understand quantum noise effects in NISQ-era optimization
- Implement error mitigation techniques for food production optimization
- Master noise modeling and quantum circuit fidelity
- Practice adaptive quantum algorithms with error correction
- Learn probabilistic solution selection under noise

## Introduction to Quantum Noise in Optimization

In real quantum computers, **noise** and **decoherence** affect optimization results:

1. **Gate errors**: Imperfect quantum operations
2. **Measurement errors**: Incorrect qubit readouts  
3. **Decoherence**: Loss of quantum information over time
4. **Crosstalk**: Unwanted interactions between qubits

**Error mitigation strategies**:
- **Zero Noise Extrapolation (ZNE)**: Extrapolate to zero noise limit
- **Readout Error Mitigation**: Correct measurement bias
- **Quantum Error Correction**: Encode logical qubits
- **Adaptive Sampling**: Increase shots for critical measurements
- **Ensemble Methods**: Average over multiple noisy runs

For **food production optimization**, noise can lead to:
- Suboptimal resource allocation
- Constraint violations
- Inconsistent solutions across runs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional, Callable
import itertools
from dataclasses import dataclass
from scipy.optimize import minimize
import networkx as nx
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Quantum Error Mitigation Tutorial Environment Ready!")
print("🔧 Available: Noise models, Error mitigation, Adaptive algorithms")

## 1. Quantum Noise Models for Food Production

Let's model how noise affects our food production optimization quantum circuits.

In [ ]:
@dataclass
class QuantumNoiseModel:
    """Models various types of quantum noise affecting optimization"""
    gate_error_rate: float = 0.01      # Single-qubit gate error probability
    two_qubit_error_rate: float = 0.05  # Two-qubit gate error probability
    readout_error_rate: float = 0.02    # Measurement error probability
    decoherence_time_t1: float = 50.0   # T1 relaxation time (μs)
    decoherence_time_t2: float = 25.0   # T2 dephasing time (μs)
    
    def apply_gate_noise(self, state: np.ndarray, gate_type: str = "single") -> np.ndarray:
        """Apply gate noise to quantum state"""
        error_rate = self.gate_error_rate if gate_type == "single" else self.two_qubit_error_rate
        
        # Simplified noise model: random bit flips with error_rate probability
        noisy_state = state.copy()
        for i in range(len(state)):
            if np.random.random() < error_rate:
                noisy_state[i] = 1 - noisy_state[i]  # Bit flip
        
        return noisy_state
    
    def apply_readout_noise(self, measurements: np.ndarray) -> np.ndarray:
        """Apply readout errors to measurement outcomes"""
        noisy_measurements = measurements.copy()
        for i in range(len(measurements)):
            if np.random.random() < self.readout_error_rate:
                noisy_measurements[i] = 1 - noisy_measurements[i]  # Readout error
        
        return noisy_measurements
    
    def calculate_fidelity(self, ideal_state: np.ndarray, noisy_state: np.ndarray) -> float:
        """Calculate quantum state fidelity"""
        # Simplified fidelity: overlap between ideal and noisy states
        overlap = np.sum(ideal_state == noisy_state) / len(ideal_state)
        return overlap

class NoisyFoodProductionQAOA:
    """QAOA for food production with realistic quantum noise"""
    
    def __init__(self, farms: List[str], foods: Dict[str, Dict], 
                 noise_model: QuantumNoiseModel, num_farms: int = 3, num_foods: int = 4):
        self.farms = farms[:num_farms]
        self.foods = {k: v for i, (k, v) in enumerate(foods.items()) if i < num_foods}
        self.noise_model = noise_model
        self.F = len(self.farms)
        self.C = len(self.foods)
        self.n_vars = self.F * self.C
        
    def create_food_qubo(self, land_constraints: Dict[str, float], 
                        target_nutrition: float = 80.0) -> np.ndarray:
        """Create QUBO matrix for food production with constraints"""
        Q = np.zeros((self.n_vars, self.n_vars))
        
        # Objective: maximize nutrition while minimizing environmental impact
        for i, farm in enumerate(self.farms):
            for j, food in enumerate(self.foods.keys()):
                var_idx = i * self.C + j
                food_data = self.foods[food]
                
                # Linear terms: nutrition benefit - environmental cost
                benefit = food_data['nutritional_value'] - 0.5 * food_data['environmental_impact']
                Q[var_idx, var_idx] = -benefit  # Negative for maximization
        
        # Land constraints: penalty for exceeding farm capacity
        penalty = 100.0
        for i, farm in enumerate(self.farms):
            land_limit = land_constraints[farm]
            
            # Add quadratic penalty for land constraint violations
            for j1 in range(self.C):
                for j2 in range(self.C):
                    var1 = i * self.C + j1
                    var2 = i * self.C + j2
                    
                    if var1 <= var2:  # Upper triangular
                        area1 = list(self.foods.values())[j1].get('min_area', 10)
                        area2 = list(self.foods.values())[j2].get('min_area', 10)
                        
                        if var1 == var2:
                            # Diagonal: individual area constraint
                            if area1 > land_limit:
                                Q[var1, var1] += penalty * area1
                        else:
                            # Off-diagonal: pairwise area constraint
                            if area1 + area2 > land_limit:
                                Q[var1, var2] += penalty * area1 * area2 / 2
        
        return Q
    
    def qaoa_expectation(self, params: np.ndarray, Q: np.ndarray, p: int = 1) -> float:
        """Calculate QAOA expectation value with noise"""
        gamma, beta = params[:p], params[p:]
        
        # Initialize uniform superposition
        n_states = 2**self.n_vars
        if self.n_vars <= 10:  # Only simulate small systems
            amplitudes = np.ones(n_states) / np.sqrt(n_states)
            
            # Apply QAOA layers with noise
            for layer in range(p):
                # Problem Hamiltonian evolution (with noise)
                for i in range(n_states):
                    bitstring = [(i >> j) & 1 for j in range(self.n_vars)]
                    cost = sum(Q[k, l] * bitstring[k] * bitstring[l] 
                              for k in range(self.n_vars) for l in range(self.n_vars))
                    amplitudes[i] *= np.exp(-1j * gamma[layer] * cost)
                
                # Apply gate noise after problem evolution
                noisy_bitstring = self.noise_model.apply_gate_noise(
                    np.array([np.random.choice([0, 1]) for _ in range(self.n_vars)]), 
                    "two_qubit"
                )
                
                # Mixer Hamiltonian evolution (X rotations with noise)
                for qubit in range(self.n_vars):
                    # Simplified mixer: add random phase noise
                    noise_factor = 1.0 + np.random.normal(0, self.noise_model.gate_error_rate)
                    for i in range(n_states):
                        if (i >> qubit) & 1:
                            amplitudes[i] *= np.exp(-1j * beta[layer] * noise_factor)
            
            # Calculate expectation value
            expectation = 0.0
            for i in range(n_states):
                prob = abs(amplitudes[i])**2
                bitstring = [(i >> j) & 1 for j in range(self.n_vars)]
                cost = sum(Q[k, l] * bitstring[k] * bitstring[l] 
                          for k in range(self.n_vars) for l in range(self.n_vars))
                expectation += prob * cost
            
            return expectation
        else:
            # For larger systems, use heuristic estimation
            return np.random.normal(0, 50)  # Simulated noisy result
    
    def optimize_with_noise_mitigation(self, Q: np.ndarray, p: int = 2, 
                                     mitigation_shots: int = 5) -> Dict:
        """Optimize with error mitigation techniques"""
        results = []
        fidelities = []
        
        # Run multiple times and average (ensemble method)
        for shot in range(mitigation_shots):
            # Random initial parameters for each shot
            initial_params = np.random.uniform(0, 2*np.pi, 2*p)
            
            # Optimize QAOA parameters
            def objective(params):
                return self.qaoa_expectation(params, Q, p)
            
            try:
                result = minimize(objective, initial_params, method='COBYLA',
                                options={'maxiter': 50})
                
                # Get optimal solution
                opt_params = result.x
                opt_value = result.fun
                
                # Extract solution bitstring (simplified)
                if self.n_vars <= 10:
                    # Sample from quantum state
                    best_bitstring = None
                    best_cost = float('inf')
                    
                    for _ in range(100):  # Sample 100 measurements
                        # Generate candidate solution
                        candidate = np.random.choice([0, 1], size=self.n_vars)
                        
                        # Apply readout noise
                        noisy_candidate = self.noise_model.apply_readout_noise(candidate)
                        
                        # Calculate cost
                        cost = sum(Q[i, j] * noisy_candidate[i] * noisy_candidate[j]
                                  for i in range(self.n_vars) for j in range(self.n_vars))
                        
                        if cost < best_cost:
                            best_cost = cost
                            best_bitstring = noisy_candidate
                
                    # Calculate fidelity (compare with ideal case)
                    ideal_solution = np.random.choice([0, 1], size=self.n_vars)  # Placeholder
                    fidelity = self.noise_model.calculate_fidelity(ideal_solution, best_bitstring)
                    
                    results.append({
                        'params': opt_params,
                        'objective': opt_value,
                        'solution': best_bitstring,
                        'cost': best_cost,
                        'fidelity': fidelity
                    })
                    fidelities.append(fidelity)
                else:
                    # For larger problems, use simplified approach
                    best_bitstring = np.random.choice([0, 1], size=self.n_vars)
                    fidelity = 0.8 + 0.2 * np.random.random()  # Simulated fidelity
                    
                    results.append({
                        'params': opt_params,
                        'objective': opt_value,
                        'solution': best_bitstring,
                        'cost': opt_value,
                        'fidelity': fidelity
                    })
                    fidelities.append(fidelity)
                    
            except Exception as e:
                print(f"Shot {shot} failed: {e}")
                continue
        
        # Error mitigation: weighted average based on fidelity
        if results:
            weights = np.array(fidelities)
            weights = weights / np.sum(weights)  # Normalize
            
            # Select best solution based on weighted probability
            selected_idx = np.random.choice(len(results), p=weights)
            best_result = results[selected_idx]
            
            return {
                'best_solution': best_result,
                'all_results': results,
                'average_fidelity': np.mean(fidelities),
                'mitigation_success': len(results) / mitigation_shots
            }
        else:
            return {'error': 'All mitigation shots failed'}

# Example usage
print("Setting up noisy food production optimization...")

# Sample food data
foods = {
    'Wheat': {'nutritional_value': 75, 'environmental_impact': 30, 'min_area': 15},
    'Rice': {'nutritional_value': 70, 'environmental_impact': 45, 'min_area': 20},
    'Corn': {'nutritional_value': 65, 'environmental_impact': 25, 'min_area': 12},
    'Soybeans': {'nutritional_value': 85, 'environmental_impact': 20, 'min_area': 18}
}

farms = ['Farm_A', 'Farm_B', 'Farm_C']
land_constraints = {'Farm_A': 50, 'Farm_B': 60, 'Farm_C': 45}

# Create noise model
noise_model = QuantumNoiseModel(
    gate_error_rate=0.02,
    two_qubit_error_rate=0.08,
    readout_error_rate=0.03
)

print(f"Noise model: Gate errors {noise_model.gate_error_rate*100:.1f}%, "
      f"Readout errors {noise_model.readout_error_rate*100:.1f}%")

## 2. Zero Noise Extrapolation (ZNE) for Food Production

ZNE estimates the zero-noise limit by running experiments at different noise levels and extrapolating.

In [ ]:
class ZeroNoiseExtrapolation:
    """Zero Noise Extrapolation for quantum food production optimization"""
    
    def __init__(self, base_noise_model: QuantumNoiseModel):
        self.base_noise = base_noise_model
        
    def create_scaled_noise_models(self, scale_factors: List[float]) -> List[QuantumNoiseModel]:
        """Create noise models with different scaling factors"""
        scaled_models = []
        
        for scale in scale_factors:
            scaled_model = QuantumNoiseModel(
                gate_error_rate=min(self.base_noise.gate_error_rate * scale, 0.5),
                two_qubit_error_rate=min(self.base_noise.two_qubit_error_rate * scale, 0.5),
                readout_error_rate=min(self.base_noise.readout_error_rate * scale, 0.5),
                decoherence_time_t1=self.base_noise.decoherence_time_t1 / scale,
                decoherence_time_t2=self.base_noise.decoherence_time_t2 / scale
            )
            scaled_models.append(scaled_model)
            
        return scaled_models
    
    def extrapolate_to_zero_noise(self, noise_scales: List[float], 
                                 objectives: List[float]) -> Tuple[float, Dict]:
        """Extrapolate objective values to zero noise limit"""
        # Fit polynomial to (noise_scale, objective) data
        if len(noise_scales) >= 2:
            # Linear extrapolation
            coeffs = np.polyfit(noise_scales, objectives, deg=1)
            zero_noise_objective = coeffs[1]  # y-intercept
            
            # Calculate R² for fit quality
            fitted_values = np.polyval(coeffs, noise_scales)
            ss_res = np.sum((objectives - fitted_values) ** 2)
            ss_tot = np.sum((objectives - np.mean(objectives)) ** 2)
            r_squared = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0
            
            extrapolation_info = {
                'coefficients': coeffs,
                'r_squared': r_squared,
                'fit_quality': 'good' if r_squared > 0.7 else 'poor',
                'noise_scales': noise_scales,
                'measured_objectives': objectives
            }
            
            return zero_noise_objective, extrapolation_info
        else:
            return objectives[0] if objectives else 0.0, {}

def run_zne_food_optimization():
    """Run ZNE-enhanced food production optimization"""
    
    # Create base noise model and ZNE object
    base_noise = QuantumNoiseModel(gate_error_rate=0.01, readout_error_rate=0.02)
    zne = ZeroNoiseExtrapolation(base_noise)
    
    # Define noise scaling factors (1.0 = base noise, higher = more noise)
    scale_factors = [1.0, 1.5, 2.0, 2.5, 3.0]
    scaled_noise_models = zne.create_scaled_noise_models(scale_factors)
    
    print("Running ZNE for food production optimization...")
    print(f"Noise scale factors: {scale_factors}")
    
    # Run optimization at each noise level
    objectives = []
    solutions = []
    
    for i, (scale, noise_model) in enumerate(zip(scale_factors, scaled_noise_models)):
        print(f"\nNoise scale {scale:.1f}: Gate error {noise_model.gate_error_rate*100:.1f}%")
        
        # Create optimizer with this noise level
        optimizer = NoisyFoodProductionQAOA(farms, foods, noise_model)
        Q = optimizer.create_food_qubo(land_constraints)
        
        # Run optimization with error mitigation
        result = optimizer.optimize_with_noise_mitigation(Q, p=1, mitigation_shots=3)
        
        if 'best_solution' in result:
            objective_value = result['best_solution']['cost']
            solution = result['best_solution']['solution']
            fidelity = result['average_fidelity']
            
            objectives.append(objective_value)
            solutions.append(solution)
            
            print(f"  Objective: {objective_value:.2f}, Fidelity: {fidelity:.3f}")
        else:
            print(f"  Optimization failed")
            objectives.append(float('inf'))
            solutions.append(None)
    
    # Perform ZNE extrapolation
    if len(objectives) >= 2 and not all(obj == float('inf') for obj in objectives):
        valid_objectives = [obj for obj in objectives if obj != float('inf')]
        valid_scales = [scale_factors[i] for i, obj in enumerate(objectives) if obj != float('inf')]
        
        zero_noise_objective, extrapolation_info = zne.extrapolate_to_zero_noise(
            valid_scales, valid_objectives)
        
        print(f"\n=== ZNE Results ===")
        print(f"Zero-noise extrapolated objective: {zero_noise_objective:.2f}")
        print(f"Fit quality (R²): {extrapolation_info.get('r_squared', 0):.3f}")
        print(f"Extrapolation quality: {extrapolation_info.get('fit_quality', 'unknown')}")
        
        # Plot ZNE results
        plt.figure(figsize=(10, 6))
        
        plt.subplot(1, 2, 1)
        plt.plot(valid_scales, valid_objectives, 'bo-', label='Measured')
        plt.axhline(y=zero_noise_objective, color='r', linestyle='--', 
                   label=f'ZNE Estimate: {zero_noise_objective:.2f}')
        plt.xlabel('Noise Scale Factor')
        plt.ylabel('Objective Value')
        plt.title('Zero Noise Extrapolation')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        # Show solution evolution with noise
        for i, sol in enumerate(solutions[:len(valid_scales)]):
            if sol is not None:
                allocation = sol.reshape(len(farms), len(foods))
                plt.imshow(allocation, cmap='viridis', alpha=0.7)
                plt.title(f'Solution at noise scale {valid_scales[i]:.1f}')
                plt.xlabel('Food Types')
                plt.ylabel('Farms')
                break  # Show only first valid solution
        
        plt.tight_layout()
        plt.show()
        
        return {
            'zne_objective': zero_noise_objective,
            'extrapolation_info': extrapolation_info,
            'noise_scales': scale_factors,
            'measured_objectives': objectives
        }
    else:
        print("ZNE failed: insufficient valid measurements")
        return None

# Run ZNE example
zne_results = run_zne_food_optimization()

## 3. Adaptive Error Mitigation with Dynamic Sampling

Adapt the number of quantum circuit shots based on solution confidence and constraint violations.

In [ ]:
class AdaptiveErrorMitigation:
    """Adaptive error mitigation for food production optimization"""
    
    def __init__(self, base_shots: int = 1024, max_shots: int = 8192):
        self.base_shots = base_shots
        self.max_shots = max_shots
        self.confidence_threshold = 0.8
        self.constraint_violation_threshold = 0.1
        
    def assess_solution_quality(self, solution: np.ndarray, Q: np.ndarray, 
                              land_constraints: Dict[str, float], 
                              farms: List[str], foods: Dict) -> Dict:
        """Assess quality of a quantum optimization solution"""
        n_farms = len(farms)
        n_foods = len(foods)
        
        # Reshape solution to farm-food allocation matrix
        allocation = solution.reshape(n_farms, n_foods)
        
        # Calculate objective value
        objective = sum(Q[i, j] * solution[i] * solution[j] 
                       for i in range(len(solution)) for j in range(len(solution)))
        
        # Check constraint violations
        violations = []
        total_violation = 0.0
        
        for i, farm in enumerate(farms):
            allocated_area = 0.0
            for j, food in enumerate(foods.keys()):
                if allocation[i, j] > 0.5:  # Binary threshold
                    min_area = list(foods.values())[j].get('min_area', 10)
                    allocated_area += min_area
            
            land_limit = land_constraints[farm]
            if allocated_area > land_limit:
                violation = (allocated_area - land_limit) / land_limit
                violations.append(violation)
                total_violation += violation
        
        # Calculate solution confidence based on objective stability
        # (simplified: assume higher absolute objective = more confident)
        confidence = min(1.0, abs(objective) / 100.0)
        
        return {
            'objective': objective,
            'constraint_violations': violations,
            'total_violation': total_violation,
            'confidence': confidence,
            'feasible': total_violation <= self.constraint_violation_threshold
        }
    
    def determine_adaptive_shots(self, solution_quality: Dict, 
                               current_shots: int) -> int:
        """Determine number of shots for next iteration based on solution quality"""
        confidence = solution_quality['confidence']
        total_violation = solution_quality['total_violation']
        
        # Increase shots if low confidence or high constraint violations
        if confidence < self.confidence_threshold or total_violation > self.constraint_violation_threshold:
            # Double shots, but don't exceed maximum
            new_shots = min(current_shots * 2, self.max_shots)
            adaptation_reason = "low_confidence" if confidence < self.confidence_threshold else "constraint_violations"
        else:
            # Keep current shots if solution is good
            new_shots = current_shots
            adaptation_reason = "stable"
        
        return new_shots, adaptation_reason
    
    def adaptive_qaoa_optimization(self, farms: List[str], foods: Dict, 
                                 land_constraints: Dict[str, float],
                                 noise_model: QuantumNoiseModel,
                                 max_iterations: int = 5) -> Dict:
        """Run QAOA with adaptive error mitigation"""
        
        # Initialize
        optimizer = NoisyFoodProductionQAOA(farms, foods, noise_model)
        Q = optimizer.create_food_qubo(land_constraints)
        
        current_shots = self.base_shots
        best_solution = None
        best_quality = None
        adaptation_history = []
        
        print("Starting adaptive error mitigation optimization...")
        print(f"Initial shots: {current_shots}")
        
        for iteration in range(max_iterations):
            print(f"\n--- Iteration {iteration + 1} ---")
            print(f"Using {current_shots} shots")
            
            # Run QAOA with current shot budget
            # Simulate shot-dependent accuracy
            shot_noise_factor = max(0.1, self.base_shots / current_shots)
            adjusted_noise = QuantumNoiseModel(
                gate_error_rate=noise_model.gate_error_rate * shot_noise_factor,
                readout_error_rate=noise_model.readout_error_rate * shot_noise_factor
            )
            
            # Create optimizer with adjusted noise
            adaptive_optimizer = NoisyFoodProductionQAOA(farms, foods, adjusted_noise)
            
            # Run optimization
            result = adaptive_optimizer.optimize_with_noise_mitigation(
                Q, p=1, mitigation_shots=1)
            
            if 'best_solution' in result:
                solution = result['best_solution']['solution']
                
                # Assess solution quality
                quality = self.assess_solution_quality(
                    solution, Q, land_constraints, farms, foods)
                
                print(f"Objective: {quality['objective']:.2f}")
                print(f"Confidence: {quality['confidence']:.3f}")
                print(f"Constraint violations: {quality['total_violation']:.3f}")
                print(f"Feasible: {quality['feasible']}")
                
                # Update best solution if this one is better
                if (best_solution is None or 
                    (quality['feasible'] and quality['objective'] < best_quality['objective'])):
                    best_solution = solution
                    best_quality = quality
                    print("  → New best solution!")
                
                # Determine adaptive shots for next iteration
                next_shots, reason = self.determine_adaptive_shots(quality, current_shots)
                
                adaptation_history.append({
                    'iteration': iteration + 1,
                    'shots': current_shots,
                    'quality': quality,
                    'next_shots': next_shots,
                    'adaptation_reason': reason
                })
                
                print(f"Next iteration shots: {next_shots} (reason: {reason})")
                current_shots = next_shots
                
                # Early stopping if solution is good enough
                if quality['confidence'] >= self.confidence_threshold and quality['feasible']:
                    print("  → Solution converged! Early stopping.")
                    break
            else:
                print("Optimization failed in this iteration")
                # Increase shots for next attempt
                current_shots = min(current_shots * 2, self.max_shots)
        
        return {
            'best_solution': best_solution,
            'best_quality': best_quality,
            'adaptation_history': adaptation_history,
            'final_shots': current_shots
        }

def demonstrate_adaptive_mitigation():
    """Demonstrate adaptive error mitigation"""
    
    # Create adaptive error mitigation system
    adaptive_em = AdaptiveErrorMitigation(base_shots=512, max_shots=4096)
    
    # Use moderate noise
    noise_model = QuantumNoiseModel(
        gate_error_rate=0.03,
        readout_error_rate=0.05
    )
    
    # Run adaptive optimization
    adaptive_result = adaptive_em.adaptive_qaoa_optimization(
        farms[:2], {k: v for i, (k, v) in enumerate(foods.items()) if i < 3},  # Smaller problem
        {f: land_constraints[f] for f in farms[:2]},
        noise_model,
        max_iterations=4
    )
    
    if adaptive_result['best_solution'] is not None:
        print(f"\n=== Adaptive Mitigation Results ===")
        print(f"Best objective: {adaptive_result['best_quality']['objective']:.2f}")
        print(f"Final confidence: {adaptive_result['best_quality']['confidence']:.3f}")
        print(f"Feasible: {adaptive_result['best_quality']['feasible']}")
        print(f"Final shot count: {adaptive_result['final_shots']}")
        
        # Plot adaptation history
        history = adaptive_result['adaptation_history']
        if history:
            iterations = [h['iteration'] for h in history]
            shots = [h['shots'] for h in history]
            confidences = [h['quality']['confidence'] for h in history]
            violations = [h['quality']['total_violation'] for h in history]
            
            plt.figure(figsize=(12, 4))
            
            plt.subplot(1, 3, 1)
            plt.plot(iterations, shots, 'bo-')
            plt.xlabel('Iteration')
            plt.ylabel('Number of Shots')
            plt.title('Adaptive Shot Count')
            plt.grid(True, alpha=0.3)
            
            plt.subplot(1, 3, 2)
            plt.plot(iterations, confidences, 'go-')
            plt.axhline(y=adaptive_em.confidence_threshold, color='r', linestyle='--', 
                       label='Threshold')
            plt.xlabel('Iteration')
            plt.ylabel('Solution Confidence')
            plt.title('Solution Confidence')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.subplot(1, 3, 3)
            plt.plot(iterations, violations, 'ro-')
            plt.axhline(y=adaptive_em.constraint_violation_threshold, color='g', linestyle='--', 
                       label='Threshold')
            plt.xlabel('Iteration')
            plt.ylabel('Constraint Violations')
            plt.title('Constraint Violations')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
    
    return adaptive_result

# Run adaptive mitigation demonstration
adaptive_results = demonstrate_adaptive_mitigation()

## 4. Ensemble Methods for Robust Food Production

Use multiple noisy quantum runs and intelligent voting/averaging to improve solution reliability.

In [ ]:
class QuantumEnsembleOptimizer:
    """Ensemble quantum optimization for robust food production planning"""
    
    def __init__(self, ensemble_size: int = 10, voting_method: str = "weighted"):
        self.ensemble_size = ensemble_size
        self.voting_method = voting_method  # "majority", "weighted", "best_fidelity"
        
    def run_ensemble_optimization(self, farms: List[str], foods: Dict,
                                land_constraints: Dict[str, float],
                                noise_models: List[QuantumNoiseModel]) -> Dict:
        """Run ensemble of quantum optimizations with different noise realizations"""
        
        ensemble_results = []
        
        print(f"Running ensemble optimization with {self.ensemble_size} members...")
        
        for i in range(self.ensemble_size):
            print(f"Ensemble member {i+1}/{self.ensemble_size}")
            
            # Use different noise model for each ensemble member
            noise_idx = i % len(noise_models)
            noise_model = noise_models[noise_idx]
            
            # Add random variation to noise parameters
            varied_noise = QuantumNoiseModel(
                gate_error_rate=noise_model.gate_error_rate * (0.8 + 0.4 * np.random.random()),
                readout_error_rate=noise_model.readout_error_rate * (0.8 + 0.4 * np.random.random())
            )
            
            # Run optimization
            optimizer = NoisyFoodProductionQAOA(farms, foods, varied_noise)
            Q = optimizer.create_food_qubo(land_constraints)
            
            result = optimizer.optimize_with_noise_mitigation(Q, p=1, mitigation_shots=2)
            
            if 'best_solution' in result:
                ensemble_results.append({
                    'solution': result['best_solution']['solution'],
                    'objective': result['best_solution']['cost'],
                    'fidelity': result['average_fidelity'],
                    'noise_model': varied_noise,
                    'member_id': i
                })
        
        # Combine ensemble results
        if ensemble_results:
            final_solution = self._combine_ensemble_solutions(ensemble_results, 
                                                            farms, foods, land_constraints)
            return final_solution
        else:
            return {'error': 'All ensemble members failed'}
    
    def _combine_ensemble_solutions(self, ensemble_results: List[Dict],
                                  farms: List[str], foods: Dict,
                                  land_constraints: Dict[str, float]) -> Dict:
        """Combine solutions from ensemble members"""
        
        if self.voting_method == "majority":
            return self._majority_voting(ensemble_results)
        elif self.voting_method == "weighted":
            return self._weighted_voting(ensemble_results, farms, foods, land_constraints)
        elif self.voting_method == "best_fidelity":
            return self._best_fidelity_selection(ensemble_results)
        else:
            raise ValueError(f"Unknown voting method: {self.voting_method}")
    
    def _majority_voting(self, ensemble_results: List[Dict]) -> Dict:
        """Combine solutions using majority voting for each variable"""
        
        # Get solution dimensions
        n_vars = len(ensemble_results[0]['solution'])
        final_solution = np.zeros(n_vars)
        
        # Vote for each variable
        for var_idx in range(n_vars):
            votes = [result['solution'][var_idx] for result in ensemble_results]
            # Majority vote (>0.5 counts as 1)
            ones_count = sum(1 for vote in votes if vote > 0.5)
            final_solution[var_idx] = 1 if ones_count > len(votes) // 2 else 0
        
        # Calculate ensemble statistics
        objectives = [r['objective'] for r in ensemble_results]
        fidelities = [r['fidelity'] for r in ensemble_results]
        
        return {
            'solution': final_solution,
            'ensemble_method': 'majority_voting',
            'ensemble_size': len(ensemble_results),
            'objective_mean': np.mean(objectives),
            'objective_std': np.std(objectives),
            'fidelity_mean': np.mean(fidelities),
            'fidelity_std': np.std(fidelities),
            'individual_results': ensemble_results
        }
    
    def _weighted_voting(self, ensemble_results: List[Dict],
                        farms: List[str], foods: Dict,
                        land_constraints: Dict[str, float]) -> Dict:
        """Combine solutions using fidelity-weighted voting"""
        
        n_vars = len(ensemble_results[0]['solution'])
        final_solution = np.zeros(n_vars)
        
        # Calculate weights based on fidelity
        fidelities = np.array([r['fidelity'] for r in ensemble_results])
        weights = fidelities / np.sum(fidelities)  # Normalize
        
        # Weighted average for each variable
        for var_idx in range(n_vars):
            weighted_sum = sum(weights[i] * ensemble_results[i]['solution'][var_idx] 
                             for i in range(len(ensemble_results)))
            final_solution[var_idx] = 1 if weighted_sum > 0.5 else 0
        
        # Calculate weighted objective
        weighted_objective = sum(weights[i] * ensemble_results[i]['objective'] 
                               for i in range(len(ensemble_results)))
        
        return {
            'solution': final_solution,
            'ensemble_method': 'weighted_voting',
            'ensemble_size': len(ensemble_results),
            'weighted_objective': weighted_objective,
            'weights': weights,
            'fidelity_mean': np.mean(fidelities),
            'individual_results': ensemble_results
        }
    
    def _best_fidelity_selection(self, ensemble_results: List[Dict]) -> Dict:
        """Select the solution with highest fidelity"""
        
        best_idx = np.argmax([r['fidelity'] for r in ensemble_results])
        best_result = ensemble_results[best_idx]
        
        return {
            'solution': best_result['solution'],
            'ensemble_method': 'best_fidelity',
            'ensemble_size': len(ensemble_results),
            'selected_member': best_idx,
            'best_fidelity': best_result['fidelity'],
            'best_objective': best_result['objective'],
            'individual_results': ensemble_results
        }

def compare_ensemble_methods():
    """Compare different ensemble voting methods"""
    
    # Create different noise models for ensemble
    noise_models = [
        QuantumNoiseModel(gate_error_rate=0.01, readout_error_rate=0.02),
        QuantumNoiseModel(gate_error_rate=0.03, readout_error_rate=0.04),
        QuantumNoiseModel(gate_error_rate=0.02, readout_error_rate=0.03)
    ]
    
    # Test different ensemble methods
    methods = ["majority", "weighted", "best_fidelity"]
    results = {}
    
    # Use smaller problem for demonstration
    small_farms = farms[:2]
    small_foods = {k: v for i, (k, v) in enumerate(foods.items()) if i < 3}
    small_constraints = {f: land_constraints[f] for f in small_farms}
    
    for method in methods:
        print(f"\n=== Testing {method} ensemble method ===")
        
        ensemble_optimizer = QuantumEnsembleOptimizer(
            ensemble_size=8, voting_method=method)
        
        result = ensemble_optimizer.run_ensemble_optimization(
            small_farms, small_foods, small_constraints, noise_models)
        
        results[method] = result
        
        if 'solution' in result:
            print(f"Final solution shape: {result['solution'].shape}")
            if 'weighted_objective' in result:
                print(f"Weighted objective: {result['weighted_objective']:.2f}")
            if 'best_objective' in result:
                print(f"Best objective: {result['best_objective']:.2f}")
            if 'fidelity_mean' in result:
                print(f"Mean fidelity: {result['fidelity_mean']:.3f}")
    
    # Visualize comparison
    if all('solution' in results[m] for m in methods):
        fig, axes = plt.subplots(2, 3, figsize=(15, 8))
        
        for i, method in enumerate(methods):
            result = results[method]
            solution = result['solution']
            
            # Plot solution
            allocation = solution.reshape(len(small_farms), len(small_foods))
            axes[0, i].imshow(allocation, cmap='viridis', aspect='auto')
            axes[0, i].set_title(f'{method.replace("_", " ").title()} Solution')
            axes[0, i].set_xlabel('Foods')
            axes[0, i].set_ylabel('Farms')
            
            # Plot ensemble statistics
            individual_results = result['individual_results']
            objectives = [r['objective'] for r in individual_results]
            fidelities = [r['fidelity'] for r in individual_results]
            
            axes[1, i].scatter(fidelities, objectives, alpha=0.7)
            axes[1, i].set_xlabel('Fidelity')
            axes[1, i].set_ylabel('Objective Value')
            axes[1, i].set_title(f'{method.replace("_", " ").title()} Ensemble')
            axes[1, i].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    return results

# Run ensemble comparison
print("Comparing ensemble methods for robust quantum optimization...")
ensemble_comparison = compare_ensemble_methods()

## 5. Real-World Error Mitigation Pipeline

Combine multiple error mitigation techniques into a comprehensive pipeline for production-ready quantum food optimization.

In [ ]:
class ProductionQuantumOptimizer:
    """Production-ready quantum optimizer with comprehensive error mitigation"""
    
    def __init__(self, mitigation_config: Dict = None):
        self.config = mitigation_config or {
            'use_zne': True,
            'use_adaptive_shots': True,
            'use_ensemble': True,
            'ensemble_size': 6,
            'base_shots': 1024,
            'max_shots': 4096,
            'zne_scale_factors': [1.0, 1.5, 2.0],
            'confidence_threshold': 0.85
        }
        
        # Initialize mitigation components
        self.zne = None
        self.adaptive_em = None
        self.ensemble_optimizer = None
        
        if self.config['use_zne']:
            base_noise = QuantumNoiseModel(gate_error_rate=0.02, readout_error_rate=0.03)
            self.zne = ZeroNoiseExtrapolation(base_noise)
        
        if self.config['use_adaptive_shots']:
            self.adaptive_em = AdaptiveErrorMitigation(
                base_shots=self.config['base_shots'],
                max_shots=self.config['max_shots']
            )
        
        if self.config['use_ensemble']:
            self.ensemble_optimizer = QuantumEnsembleOptimizer(
                ensemble_size=self.config['ensemble_size'],
                voting_method="weighted"
            )
    
    def optimize_food_production(self, farms: List[str], foods: Dict,
                                land_constraints: Dict[str, float],
                                nutrition_targets: Dict[str, float] = None) -> Dict:
        """Run comprehensive quantum optimization with all error mitigation techniques"""
        
        print("=== Production Quantum Food Optimization ===")
        print(f"Farms: {len(farms)}, Foods: {len(foods)}")
        print(f"Mitigation techniques: {[k for k, v in self.config.items() if k.startswith('use_') and v]}")
        
        optimization_results = {
            'config': self.config,
            'problem_size': {'farms': len(farms), 'foods': len(foods)},
            'mitigation_results': {}
        }
        
        # Step 1: Ensemble optimization with multiple noise realizations
        if self.config['use_ensemble']:
            print("\n--- Step 1: Ensemble Optimization ---")
            
            # Create diverse noise models for ensemble
            noise_models = [
                QuantumNoiseModel(gate_error_rate=0.015, readout_error_rate=0.025),
                QuantumNoiseModel(gate_error_rate=0.025, readout_error_rate=0.035),
                QuantumNoiseModel(gate_error_rate=0.020, readout_error_rate=0.030)
            ]
            
            ensemble_result = self.ensemble_optimizer.run_ensemble_optimization(
                farms, foods, land_constraints, noise_models)
            
            optimization_results['mitigation_results']['ensemble'] = ensemble_result
            
            if 'solution' in ensemble_result:
                print(f"Ensemble complete: {ensemble_result['ensemble_size']} members")
                print(f"Mean fidelity: {ensemble_result.get('fidelity_mean', 0):.3f}")
        
        # Step 2: Zero Noise Extrapolation (if applicable)
        if self.config['use_zne'] and len(farms) * len(foods) <= 12:  # ZNE for smaller problems
            print("\n--- Step 2: Zero Noise Extrapolation ---")
            
            zne_noise_models = self.zne.create_scaled_noise_models(
                self.config['zne_scale_factors'])
            
            zne_objectives = []
            for scale, noise_model in zip(self.config['zne_scale_factors'], zne_noise_models):
                optimizer = NoisyFoodProductionQAOA(farms, foods, noise_model)
                Q = optimizer.create_food_qubo(land_constraints)
                result = optimizer.optimize_with_noise_mitigation(Q, p=1, mitigation_shots=2)
                
                if 'best_solution' in result:
                    zne_objectives.append(result['best_solution']['cost'])
                    print(f"  Scale {scale}: Objective {result['best_solution']['cost']:.2f}")
                else:
                    zne_objectives.append(float('inf'))
            
            # Perform extrapolation
            zne_objective, zne_info = self.zne.extrapolate_to_zero_noise(
                self.config['zne_scale_factors'], zne_objectives)
            
            optimization_results['mitigation_results']['zne'] = {
                'zero_noise_objective': zne_objective,
                'extrapolation_info': zne_info
            }
            
            print(f"ZNE zero-noise objective: {zne_objective:.2f}")
            print(f"Fit quality (R²): {zne_info.get('r_squared', 0):.3f}")
        
        # Step 3: Adaptive shots refinement
        if self.config['use_adaptive_shots']:
            print("\n--- Step 3: Adaptive Shots Refinement ---")
            
            # Use moderate noise for adaptive refinement
            adaptive_noise = QuantumNoiseModel(gate_error_rate=0.02, readout_error_rate=0.03)
            
            adaptive_result = self.adaptive_em.adaptive_qaoa_optimization(
                farms, foods, land_constraints, adaptive_noise, max_iterations=3)
            
            optimization_results['mitigation_results']['adaptive'] = adaptive_result
            
            if adaptive_result['best_solution'] is not None:
                print(f"Adaptive optimization complete")
                print(f"Final confidence: {adaptive_result['best_quality']['confidence']:.3f}")
                print(f"Final shot count: {adaptive_result['final_shots']}")
        
        # Step 4: Solution integration and validation
        print("\n--- Step 4: Solution Integration ---")
        
        final_solution = self._integrate_solutions(optimization_results, farms, foods, land_constraints)
        optimization_results['final_solution'] = final_solution
        
        # Validate final solution
        validation = self._validate_solution(final_solution, farms, foods, land_constraints)
        optimization_results['validation'] = validation
        
        print(f"Final solution validation:")
        print(f"  Feasible: {validation['feasible']}")
        print(f"  Objective: {validation['objective']:.2f}")
        print(f"  Constraint violations: {validation['max_violation']:.3f}")
        
        return optimization_results
    
    def _integrate_solutions(self, optimization_results: Dict, 
                           farms: List[str], foods: Dict,
                           land_constraints: Dict[str, float]) -> np.ndarray:
        """Integrate solutions from different mitigation techniques"""
        
        solutions = []
        weights = []
        
        # Collect solutions and their confidence weights
        mitigation_results = optimization_results['mitigation_results']
        
        # Ensemble solution
        if 'ensemble' in mitigation_results and 'solution' in mitigation_results['ensemble']:
            ensemble_sol = mitigation_results['ensemble']['solution']
            ensemble_weight = mitigation_results['ensemble'].get('fidelity_mean', 0.5)
            solutions.append(ensemble_sol)
            weights.append(ensemble_weight)
        
        # Adaptive solution
        if 'adaptive' in mitigation_results and mitigation_results['adaptive']['best_solution'] is not None:
            adaptive_sol = mitigation_results['adaptive']['best_solution']
            adaptive_weight = mitigation_results['adaptive']['best_quality']['confidence']
            solutions.append(adaptive_sol)
            weights.append(adaptive_weight)
        
        # If we have multiple solutions, use weighted averaging
        if len(solutions) > 1:
            weights = np.array(weights) / np.sum(weights)  # Normalize
            
            n_vars = len(solutions[0])
            integrated_solution = np.zeros(n_vars)
            
            for var_idx in range(n_vars):
                weighted_value = sum(weights[i] * solutions[i][var_idx] for i in range(len(solutions)))
                integrated_solution[var_idx] = 1 if weighted_value > 0.5 else 0
            
            return integrated_solution
        elif len(solutions) == 1:
            return solutions[0]
        else:
            # Fallback: random solution
            return np.random.choice([0, 1], size=len(farms) * len(foods))
    
    def _validate_solution(self, solution: np.ndarray, farms: List[str], 
                          foods: Dict, land_constraints: Dict[str, float]) -> Dict:
        """Validate the final integrated solution"""
        
        n_farms = len(farms)
        n_foods = len(foods)
        allocation = solution.reshape(n_farms, n_foods)
        
        # Check feasibility
        violations = []
        total_objective = 0.0
        
        for i, farm in enumerate(farms):
            allocated_area = 0.0
            farm_objective = 0.0
            
            for j, (food_name, food_data) in enumerate(foods.items()):
                if allocation[i, j] > 0.5:
                    min_area = food_data.get('min_area', 10)
                    allocated_area += min_area
                    
                    # Calculate contribution to objective
                    benefit = food_data['nutritional_value'] - 0.5 * food_data['environmental_impact']
                    farm_objective += benefit
            
            total_objective += farm_objective
            
            # Check land constraint
            land_limit = land_constraints[farm]
            if allocated_area > land_limit:
                violation = (allocated_area - land_limit) / land_limit
                violations.append(violation)
        
        max_violation = max(violations) if violations else 0.0
        
        return {
            'feasible': max_violation <= 0.1,  # 10% tolerance
            'objective': total_objective,
            'max_violation': max_violation,
            'num_violations': len(violations),
            'allocation_matrix': allocation
        }

def run_production_optimization():
    """Run production-level quantum optimization with full error mitigation"""
    
    # Configure production optimizer
    config = {
        'use_zne': True,
        'use_adaptive_shots': True,
        'use_ensemble': True,
        'ensemble_size': 5,
        'base_shots': 512,
        'max_shots': 2048,
        'zne_scale_factors': [1.0, 1.5, 2.0],
        'confidence_threshold': 0.8
    }
    
    optimizer = ProductionQuantumOptimizer(config)
    
    # Use smaller problem for demonstration
    prod_farms = farms[:2]
    prod_foods = {k: v for i, (k, v) in enumerate(foods.items()) if i < 3}
    prod_constraints = {f: land_constraints[f] for f in prod_farms}
    
    # Run optimization
    results = optimizer.optimize_food_production(
        prod_farms, prod_foods, prod_constraints)
    
    # Visualize results
    if 'final_solution' in results:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # Final allocation
        allocation = results['validation']['allocation_matrix']
        im1 = axes[0, 0].imshow(allocation, cmap='viridis', aspect='auto')
        axes[0, 0].set_title('Final Allocation')
        axes[0, 0].set_xlabel('Foods')
        axes[0, 0].set_ylabel('Farms')
        plt.colorbar(im1, ax=axes[0, 0])
        
        # Mitigation techniques used
        techniques = [k for k, v in config.items() if k.startswith('use_') and v]
        y_pos = np.arange(len(techniques))
        axes[0, 1].barh(y_pos, [1]*len(techniques), color='skyblue')
        axes[0, 1].set_yticks(y_pos)
        axes[0, 1].set_yticklabels([t.replace('use_', '').replace('_', ' ').title() for t in techniques])
        axes[0, 1].set_title('Mitigation Techniques Used')
        
        # Ensemble results (if available)
        if 'ensemble' in results['mitigation_results']:
            ensemble_data = results['mitigation_results']['ensemble']
            if 'individual_results' in ensemble_data:
                individual = ensemble_data['individual_results']
                objectives = [r['objective'] for r in individual]
                fidelities = [r['fidelity'] for r in individual]
                
                axes[1, 0].scatter(fidelities, objectives, alpha=0.7, color='orange')
                axes[1, 0].set_xlabel('Fidelity')
                axes[1, 0].set_ylabel('Objective Value')
                axes[1, 0].set_title('Ensemble Member Performance')
                axes[1, 0].grid(True, alpha=0.3)
        
        # Validation metrics
        validation = results['validation']
        metrics = ['Objective', 'Max Violation', 'Feasible']
        values = [validation['objective'], validation['max_violation'], float(validation['feasible'])]
        
        bars = axes[1, 1].bar(metrics, values, color=['green', 'red', 'blue'])
        axes[1, 1].set_title('Solution Validation')
        axes[1, 1].set_ylabel('Value')
        
        # Add value labels on bars
        for bar, value in zip(bars, values):
            height = bar.get_height()
            axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                           f'{value:.2f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
    
    return results

# Run production optimization
print("Running production-level quantum optimization with comprehensive error mitigation...")
production_results = run_production_optimization()

## Summary and Real-World Applications

### Key Error Mitigation Techniques Covered:

1. **Quantum Noise Models**: Realistic modeling of gate errors, readout errors, and decoherence
2. **Zero Noise Extrapolation (ZNE)**: Extrapolating to ideal quantum performance
3. **Adaptive Sampling**: Dynamic adjustment of quantum circuit shots based on solution quality
4. **Ensemble Methods**: Combining multiple noisy quantum runs for robust solutions
5. **Integrated Pipeline**: Production-ready system combining all techniques

### Applications in Food Production:

- **Resource Allocation**: Robust farm-food assignments under quantum noise
- **Constraint Handling**: Reliable land use constraint satisfaction
- **Multi-objective Optimization**: Balanced nutrition, sustainability, and cost optimization
- **Uncertainty Management**: Handling quantum and agricultural uncertainties

### Next Steps:

1. **Hardware-Specific Calibration**: Adapt noise models to specific quantum devices
2. **Real-Time Adaptation**: Dynamic error mitigation based on live quantum hardware performance
3. **Hybrid Classical-Quantum**: Combine quantum error mitigation with classical optimization
4. **Scalability**: Error mitigation for larger food production networks

### Performance Considerations:

- **Circuit Depth**: Shorter circuits are more noise-resilient
- **Shot Budget**: Balance between accuracy and computational cost
- **Problem Decomposition**: Break large problems into noise-resilient subproblems
- **Error Threshold**: Define acceptable error rates for different optimization stages

The techniques demonstrated here provide a foundation for deploying quantum optimization in real food production systems where reliability and robustness are critical.